# Vmax - Rmax

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import matplotlib

In [2]:
DATA_PATH = "../data/TangoSIDM/"

IDs = {
    "SigmaConstant00": "CDM",
    "SigmaConstant01": "SigmaConstant01",
    "SigmaConstant10": "SigmaConstant10",
    "SigmaVelDep20Anisotropic": "SigmaVel20",
    "SigmaVelDep60Anisotropic": "SigmaVel60", 
    "SigmaVelDep100Anisotropic": "SigmaVel100",
}

In [3]:
for id, id_name in IDs.items():
    print(f"\n \n{id_name}")
    c_res = 0
    c_outside = 0
    c_tot = 0
    file = h5py.File(DATA_PATH+f"{id}.hdf5", "r")
    for idx in file.keys():
        halo_data = file[f'{idx}']["halo_data"]
        density = halo_data["dm_density"]
        centers = halo_data["centered_radial_bins"]
        rotation = halo_data["dm_rotation"][:]
        centers_rotation = halo_data["centered_rotation_radial_bins"][:]
        tree_data = file[f'{idx}']["tree_data"]
        Vmax = tree_data["Vmax"][0]   
        
        idx = np.abs(rotation - Vmax).argmin()
        Rmax = centers_rotation[idx] 
        c_tot+=1
        if Rmax < 0.650: c_res+1

        if np.abs(rotation - Vmax).min()>1:
            c_outside +=1
            # plt.figure(figsize=(4,4))
            # plt.plot(centers_rotation, rotation)
            # plt.scatter(Rmax, Vmax)
            # plt.axvline(0.650, c='k')
            # plt.xscale("log")
            # plt.xlabel("Radius [kpc]")
            # plt.ylabel("Circular velocity [km/s]")
            # plt.tick_params(direction='in', axis='both', which='both', pad=4.5)
            # plt.tight_layout()
            # plt.show()
        
    print(f"Excluded {c_res} / {c_tot}")
    print(f"Outside {c_outside}/ {c_tot}")
    file.close()


 
CDM
Excluded 0 / 714
Outside 40/ 714

 
SigmaConstant01
Excluded 0 / 693
Outside 32/ 693

 
SigmaConstant10
Excluded 0 / 651
Outside 33/ 651

 
SigmaVel20
Excluded 0 / 693
Outside 36/ 693

 
SigmaVel60
Excluded 0 / 693
Outside 34/ 693

 
SigmaVel100
Excluded 0 / 693
Outside 34/ 693


Add Rmax at z=0 data to datasets

In [4]:
for id, id_name in IDs.items():
    print(f"\n \n{id_name}")
    c_ok = 0
    c_tot = 0
    file = h5py.File(DATA_PATH+f"{id}.hdf5", "a")
    for idx in file.keys():
        c_tot+=1
        halo_data = file[f'{idx}']["halo_data"]
        rotation = halo_data["dm_rotation"][:]
        centers_rotation = halo_data["centered_rotation_radial_bins"][:]
        tree_data = file[f'{idx}']["tree_data"]
        Vmax = tree_data["Vmax"][0]   
        
        idx = np.abs(rotation - Vmax).argmin()
        Rmax = centers_rotation[idx] 

        if np.abs(rotation - Vmax).min()<1:
            c_ok +=1
            tree_data.require_dataset(f"Rmax", data=Rmax, shape=Rmax.shape, dtype=Rmax.dtype)
       
    print(f"Rmax included for {c_ok} / {c_tot}")
    file.close()


 
CDM
Rmax included for 674 / 714

 
SigmaConstant01
Rmax included for 661 / 693

 
SigmaConstant10
Rmax included for 618 / 651

 
SigmaVel20
Rmax included for 657 / 693

 
SigmaVel60
Rmax included for 659 / 693

 
SigmaVel100
Rmax included for 659 / 693
